In [1]:
import pyopenms as oms

def load_file(mzmL_path:str):
    exp=oms.MSExperiment()
    oms.MzMLFile().load(mzmL_path,exp)
    return exp 

exp=load_file(mzmL_path="/home/crs/dev/peroxide/data/2.mzML")

RuntimeError: the file '/home/crs/dev/peroxide/data/2.mzML' could not be found

In [ ]:
import numpy as np

snr_threshold = 3
intensity_list = []

for i, spec in enumerate(exp):
    current_intensities = {}
    if spec.getMSLevel() != 1:
        continue
    
    rt = spec.getRT()
    mzs, intensities = spec.get_peaks()
    
    median_intensity = np.median(intensities)
    mad = np.median(np.abs(intensities - median_intensity))
    noise_sigma = 1.4826 * mad
    if noise_sigma <= 0:
        continue
    
    sn = intensities / noise_sigma
    mask = (sn >= snr_threshold)
    for mz, inten in zip(mzs[mask], intensities[mask]):
        current_intensities[mz] = inten
    intensity_list.append(current_intensities)

In [ ]:
import numpy as np

ppm_tolerance = 10
zero_threshold = 0.99

all_mz = []
for frame in intensity_list:
    all_mz.extend(frame.keys())
all_mz = np.sort(np.array(all_mz))
    
merged_mz = [all_mz[0]]
current_sum = all_mz[0]
current_count = 1
    
for mz in all_mz[1:]:
    avg_mz = current_sum / current_count
    ppm_diff = abs(mz - avg_mz) / avg_mz * 1e6
    if ppm_diff <= ppm_tolerance:
        current_sum += mz
        current_count += 1
        merged_mz[-1] = current_sum / current_count
    else:
        merged_mz.append(mz)
        current_sum = mz
        current_count = 1

mz_axis = np.array(merged_mz)

intensity_matrix = np.zeros((len(intensity_list), len(mz_axis)))

for i, frame in enumerate(intensity_list):
    mzs = np.array(list(frame.keys()))
    intens = np.array(list(frame.values()))
    idxs = np.searchsorted(mz_axis, mzs)
    for mz, inten, idx in zip(mzs, intens, idxs):
        for neighbor in (idx-1, idx, idx+1):
            if 0 <= neighbor < len(mz_axis):
                if abs(mz - mz_axis[neighbor]) / mz * 1e6 <= ppm_tolerance:
                    intensity_matrix[i, neighbor] = inten
                    break
                
zero_ratio = np.mean(intensity_matrix == 0, axis=0)
keep_cols = zero_ratio < zero_threshold
intensity_matrix = intensity_matrix[:, keep_cols]
mz_axis = mz_axis[keep_cols]

In [ ]:
import pandas as pd
intensity_df = pd.DataFrame(intensity_matrix, columns=mz_axis)
intensity_df.to_csv("4.csv", index=False)

In [ ]:
target_mz = 758.5685
mz_axis = np.array(mz_axis)
ppm_diffs = np.abs((mz_axis - target_mz) / mz_axis * 1e6)
nearest_idx = np.argmin(ppm_diffs)
nearest_mz = mz_axis[nearest_idx]
mz_intensity = np.array(intensity_matrix)[:, nearest_idx]

In [ ]:
from umap import UMAP
umap_model = UMAP(n_neighbors=10, min_dist=0.1, n_components=2, random_state=42)
embedding = umap_model.fit_transform(intensity_matrix)

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
scaled_data = StandardScaler().fit_transform(intensity_matrix)
dbscan_model = DBSCAN(eps=2, min_samples=5)
dbscan_model.fit(scaled_data)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(embedding[:, 0], embedding[:, 1], c=mz_intensity, cmap='viridis', s=1)
plt.title("UMAP Projection of MS1 Spectra")
plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
print(dbscan_model.labels_)
plt.scatter(embedding[:, 0], embedding[:, 1], c=dbscan_model.labels_, s=1)
plt.title("UMAP Projection of MS1 Spectra")
plt.axis('off')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
X = intensity_df.values
X_centered = X - X.mean(axis=0)
pca = PCA(n_components=2)
scores = pca.fit_transform(X_centered)
loadings = pca.components_.T
import matplotlib.pyplot as plt
plt.scatter(scores[:, 0], scores[:, 1], label='PC1 Scores', linewidth=1)
plt.title('PCA Scores')
plt.legend()
plt.savefig("pca_scores.svg")

## 细胞帧提取方案一

使用上面整理得到的质谱数据矩阵$X\in\mathbb{R}^{t\times m}$，按行中心化，即取一个平均帧$\boldsymbol{x}_0^T=\frac{1}{t}\sum\limits_{i=0}^t\boldsymbol{x}_i^T$，给$X$的每一行$\boldsymbol{x}_i^T$都减去这个平均帧，然后使用一个秩一矩阵逼近它，即找到向量$\boldsymbol{a}\in\mathbb{R}^{t}$和向量$\boldsymbol{b}\in\mathbb{R}^m$，使得

$$\min_{\boldsymbol{a}\geq0, \boldsymbol{b}\geq0}\|X-\boldsymbol{a}\boldsymbol{b}^T\|_F^2$$

其中$\boldsymbol{a}$表示数据的时序特征，序列$\boldsymbol{a}$上的峰值表示细胞帧；$\boldsymbol{b}$是m/z的特征，其序列上的峰值表示细胞代谢物。

目标函数$$J(\boldsymbol{a},\boldsymbol{b})=\|X-\boldsymbol{a}\boldsymbol{b}^T\|_F^2$$的梯度
$$\frac{\partial J}{\partial\boldsymbol{a}}=-2X\boldsymbol{b}+2\|\boldsymbol{b}\|^2\boldsymbol{a}$$

$$\frac{\partial J}{\partial\boldsymbol{b}}=-2X^T\boldsymbol{a}+2\|\boldsymbol{a}\|^2\boldsymbol{b}$$

迭代优化时，固定$\boldsymbol{a}$，更新$\boldsymbol{b}$为目标函数取极小值且满足非负约束时对应的值，反之亦然。即：
$$\boldsymbol{a}^\text{new}=\max(0,\frac{X\boldsymbol{b}}{\|\boldsymbol{b}\|^2})$$

$$\boldsymbol{b}^\text{new}=\max(0,\frac{X^T\boldsymbol{a}}{\|\boldsymbol{a}\|^2})$$

In [ ]:
def rank1_estimation(intensities, tol:float=1e-6, max_iter = 100):
    a, b = np.ones((intensities.shape[0], 1)), np.ones((intensities.shape[1], 1))
    for _ in range(max_iter):
        a_new = intensities @ b / (b.T @ b)
        b_new = intensities.T @ a_new / (a_new.T @ a_new)
        if np.linalg.norm(a_new - a) < tol and np.linalg.norm(b_new - b) < tol:
            break
        a, b = a_new, b_new
    return a, b

In [ ]:
a, b = rank1_estimation(intensity_df.values)
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(a, label='Frame scores', linewidth=1)
ax[0].set_title('Frame Scores')
ax[1].plot(intensity_df.columns, b, label='m/z loadings', linewidth=1)
ax[1].set_title('m/z Loadings')
plt.savefig("rank1_estimation.svg")
print(intensity_df.columns.values[np.argsort(-b.flatten())[:10]])

In [ ]:
from sklearn.decomposition import NMF
n_components = 3

nmf_model = NMF(n_components)
T = nmf_model.fit_transform(intensity_df.values)
M = nmf_model.components_

fig, ax = plt.subplots(2, 3, figsize=(12, 6))

for i in range(3):
    ax[0, i].plot(T[:, i], label=f'Component {i+1}')
    ax[0, i].set_title(f'Frame profile {i+1}')
    ax[0, i].set_xlabel("Frame index")
    ax[0, i].set_ylabel("Relative abundance")
    ax[0, i].legend()

for i in range(3):
    ax[1, i].plot(intensity_df.columns.values, M[i], label=f'Component {i+1}')
    ax[1, i].set_title(f'm/z profile {i+1}')
    ax[1, i].set_xlabel("m/z")
    ax[1, i].set_ylabel("Intensity")
    ax[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
intensities = intensity_df.values
med_intensity = np.median(intensities, axis=0, keepdims=True)
intensities_1 = intensities - med_intensity

import matplotlib.pyplot as plt
mz = 734.5929
mz_list = intensity_df.columns.values
nearest_idx = np.argmin(np.abs(mz_list - mz))
nearest_mz = mz_list[nearest_idx]
print(nearest_idx, nearest_mz, med_intensity[0][nearest_idx], np.max(intensities_1[:,nearest_idx]))
print(intensities_1.sum())
plt.plot(intensities_1[:,nearest_idx], linewidth = 1)
plt.show()

In [ ]:
kernels = [
    np.array([-1, 1]),  
    #np.array([-1, 0, 1, -1]),
    #np.array([-1, 0, 0, 1, -1])
]
scores = [np.convolve(intensities[:,nearest_idx], k, mode='same') for k in kernels]
score = np.max(np.stack(scores), axis=0)
pulse_positions = np.where(score < np.percentile(score, 5))[0]
plt.plot(intensities[:,nearest_idx], linewidth = 1)
plt.scatter(pulse_positions, intensities[pulse_positions, nearest_idx], color = "r", s = 3)
plt.show()

In [ ]:
from scipy.ndimage import median_filter
intensities = intensity_df.values
baseline = median_filter(intensities, size = (20,1))
fig, ax = plt.subplots(2,1)
ax[0].plot(intensities[:,nearest_idx], linewidth = 1)
ax[0].plot(baseline[:,nearest_idx], linewidth = 1, color = "r")
ax[1].plot((intensities - baseline)[:,nearest_idx])
plt.savefig(f"baseline_reg_at_{nearest_mz}.svg")
plt.show()

# 细胞帧提取方案二——无监督的CyESI信号重构方案
假设CyESI的信号$s_{m,t}$可以这样进行分解：
$$s_{m,t} = b_{m,t} + \sigma_m\epsilon + c_{m,t}$$
其中$b_{m,t}$是基线信号，通过中位数滤波器获得；$\sigma$是背景噪音的标准差，$\epsilon\sim\mathcal{N}(0,1)$；$c_{m,t}$是细胞脉冲信号，优化时受恒正约束和稀疏正则（我们希望细胞信号脉冲是正相的，且只在少数时间帧出现）。那么重构信号的目标就是在保证$c_{m,t}\geq0$的前提下最小化损失函数
$$\mathcal{L} = \sum_{m,t}\bigg(\frac{(s_{m,t}-b_{m,t}-c_{m,t})^2}{2\sigma_m^2}+\log\sigma_m+\lambda\frac{c_{m,t}}{\sigma_m}\bigg)$$
前两项来源于$s_{m,t}$的最大似然估计，第三项是$c_{m,t}$的L1正则。将$\sigma_m$初始化为$r_{m,t}=s_{m,t}-b_{m,t}$的标准差，$c_{m}$初始化为$\boldsymbol{0}_t$，使用交替优化的方式迭代更新$\sigma$和$c$，分别求$\mathcal{L}$对$c_{m,t}$和$\sigma_m$的偏导得
$$\frac{\partial\mathcal{L}}{\partial c_{m,t}}=-\frac{s_{m,t}-b_{m,t}-c_{m,t}}{\sigma_m^2}+\frac{\lambda}{\sigma_m}$$
$$\frac{\partial\mathcal{L}}{\partial\sigma_m}=\sum_t\bigg(-\frac{(s_{m,t}-b_{m,t}-c_{m,t})^2}{\sigma_m^3}-\frac{\lambda c_{m,t}}{\sigma_m^2}+\frac{1}{\sigma_m}\bigg)$$
令偏导数为0，并用软阈值确保$c_{m,t}$的稀疏性，得到
$$\tilde{c}_{m,t}=r_{m,t}-\lambda\sigma_m,\space c_{m,t}=\max(\tilde{c}_{m,t} - \tau\sigma_m,0)$$
$$\sigma=\frac{\lambda C+\sqrt{(\lambda C)^2+4TR}}{2T}$$
其中$\tau$是用户指定的用于限制$c_{m,t}$稀疏性的超参，$R=\sum\limits_t(s_{m,t}-b_{m,t}-c_{m,t})^2$，$C=\sum\limits_t c_{m,t}$，$T$是时间帧的总数。

In [ ]:
def optimize_single_channel(s, b, lam=0.5, sigma_min=1e-3, max_iter=50, tol=1e-4):
    
    r = s - b
    sigma = np.maximum(np.std(r), sigma_min)
    c = np.zeros_like(r)

    for _ in range(max_iter):
        c_new = np.maximum(r - lam * sigma, 0.0)
        residual = s - b - c_new
        sum_c = np.sum(c_new)
        t = len(r)
        term = lam * sum_c
        sigma_new = (term + np.sqrt(term ** 2 + 4 * t * np.sum(residual))) / (2 * t)
        sigma_new = max(sigma_new, sigma_min)

        diff = np.linalg.norm(c_new - c) / (np.linalg.norm(c) + 1e-12)
        if diff < tol:
            c = c_new
            sigma = sigma_new
            break

        c = c_new
        sigma = sigma_new

    return c, sigma

def sparse_reconstruction_parallel(S, B, lam=0.5, sigma_min=1e-3, max_iter=50, n_jobs=-1):
    from joblib import Parallel, delayed
    from tqdm import tqdm
    T, M = S.shape
    results = Parallel(n_jobs = n_jobs)(
        delayed(optimize_single_channel)(S[:, m], B[:, m], lam, sigma_min, max_iter)
        for m in tqdm(range(M), desc="Parallel reconstruction")
    )

    C = np.column_stack([r[0] for r in results])
    sigma = np.array([r[1] for r in results])
    return C, sigma

In [ ]:
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(10, 6))
gs = gridspec.GridSpec(2, 2, width_ratios=[2, 1], height_ratios=[1, 1])
ax_large = fig.add_subplot(gs[:, 0])
ax_top = fig.add_subplot(gs[0, 1])
ax_bottom = fig.add_subplot(gs[1, 1])

mz = 734.5929
mz_list = intensity_df.columns.values
nearest_idx = np.argmin(np.abs(mz_list - mz))
nearest_mz = mz_list[nearest_idx]
C, sigma = sparse_reconstruction_parallel(intensities, baseline, lam = 5)
c_df = pd.DataFrame(C, columns=mz_axis)
c_df.to_csv("c.csv", index=False)
ax_large.plot(intensities[:,nearest_idx])
ax_large.plot(baseline[:,nearest_idx], color = "r")
ax_large.plot(C[:,nearest_idx], color="yellow")
ax_top.plot(mz_axis, np.sum(C, axis = 0))
ax_bottom.plot(np.sum(np.nan_to_num(C, nan = 0), axis = 1))
plt.show()

In [ ]:
a, b = rank1_estimation(np.nan_to_num(C, nan = 0.0))
med_a = np.median(a)
snr = 40
idx = np.where(a - med_a > snr * med_a)[0]
denoised = pd.DataFrame(np.concatenate([idx.reshape(len(idx), 1), intensities[idx, :]], axis = 1), 
                        columns = np.concatenate([[0], mz_axis]))
denoised.to_csv("denoised.csv", index = False)
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(a, label='Frame scores', linewidth=1)
ax[0].set_title('Frame Scores')
ax[1].plot(mz_axis, b, label='m/z loadings', linewidth=1)
ax[1].set_title('m/z Loadings')
plt.savefig("rank1_estimation_c.svg")
print(intensity_df.columns.values[np.argsort(-b.flatten())[:10]])